In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas
import researchpy as rp
import scipy.stats as stats
import statsmodels.api as sm
import os
import yaml
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing
import dataframe_image as dfi

In [40]:
def calcStuff(name, df, method):            
    subset = df[['inliers', 'sharpness', 'flow', 'contrast', 'average intensity', 'percentage white pixels', 'percentage black pixels']]
    calcCorrelation(subset,name, plots_dir, method)
    

In [41]:
def makePlots(subset_scaled, name, directory):
    plt = subset_scaled.plot(xlabel='frame number', title=name)
    fig = plt.get_figure()
    fig.savefig(directory+'/plots/'+name+'.png')
    

Correlation coeffs

In [66]:
def calcCorrelation(subset_scaled, name, directory, method):
    corr = subset_scaled.corr(method='pearson').style.background_gradient(cmap='coolwarm')
    corr.set_caption(name)
    if method == 'train':
        dfi.export(corr, directory+'/corr_plots/'+name+'_corr_train.png')
    elif method == 'test':
        dfi.export(corr, directory+'/corr_plots/'+name+'_corr_test.png')

In [67]:
#directory = 'matching_interframe'
directory = 'matching_interframe'
plots_dir = os.getcwd()
extension = ('.csv')
filepaths = []
for subdir, dirs, files in os.walk(directory):
    files = sorted(files)
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(extension):
            filepaths.append(filepath)
            '''
            df = pandas.read_csv(filepath)
            df = df.astype(float)
            df.rename(columns = {'blur':'sharpness'}, inplace=True)
            df.dropna(inplace=True)
            df_shuffled = df.sample(frac=1.0, replace=False, random_state=1)
            #df_shuffled = df
            midpoint = int(df_shuffled.shape[0]/2)
            train_df = df_shuffled.iloc[0:midpoint]
            test_df = df_shuffled.iloc[-midpoint:]
            name = (filepath.split('/',1)[1]).split('_stats.csv')[0].replace('/', '_')
            calcStuff(name+ ' train', train_df, method='train')
            calcStuff(name+ ' test', test_df, method='test')
            '''
df = pandas.concat([pandas.read_csv(f) for f in filepaths])
df = df.astype(float)
df.rename(columns = {'blur':'sharpness'}, inplace=True)
df.dropna(inplace=True)
name = directory.split('/')[-1]
name='all_combined'
#df_shuffled = df.sample(frac=1.0, replace=False, random_state=1)
df_shuffled = df
midpoint = int(df_shuffled.shape[0]/2)
train_df = df_shuffled.iloc[0:midpoint]
test_df = df_shuffled.iloc[-midpoint:]
calcStuff(name+ ' train', train_df, method='train')
calcStuff(name+ ' test', test_df, method='test')


[0816/144657.741763:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0816/144657.752184:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0816/144657.940403:INFO:headless_shell.cc(660)] Written to file /tmp/tmp1cvducib/temp.png.
[0816/144658.204177:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0816/144658.214454:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0816/144658.408887:INFO:headless_shell.cc(660)] Written to file /tmp/tmpi4xtfqvf/temp.png.


In [44]:
file = "matching_interframe/flourish/DatasetA_stats.csv"

#file='matching_interframe/euroc/MH01_stats.csv'
df = pandas.read_csv(file)
df.rename(columns = {'blur':'sharpness'}, inplace=True)
midpoint = int(df.shape[0]/2)
train_df = df.iloc[0:midpoint]
test_df = df.iloc[-midpoint:]
subset = df[['inliers', 'sharpness', 'flow', 'contrast', 'average intensity', 'percentage white pixels', 'percentage black pixels']]
#corr = subset.corr(method='spearman')
#corr.style.background_gradient(cmap='coolwarm')
#plt.show()
figure()
fig, plot = plt.subplots(2, figsize = (6,6))
plot[0].scatter(train_df['percentage black pixels'], train_df['inliers'])
plot[1].scatter(test_df['percentage black pixels'], test_df['inliers'])

NameError: name 'figure' is not defined